In [1]:
!pip install groq python-dotenv numpy tqdm datasets

  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.41.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.3 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
Using cached pydantic-2.12.5-py3-none-any.whl (463 kB)
Using cached pydantic_core-2.41.5-cp312-cp312-macosx_11_0_arm64.whl (1.9 MB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached typing_inspection-0.4.2-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [groq]6/7 [groq]


In [3]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

Generating test split: 100%|██████████| 1319/1319 [00:00<00:00, 532257.74 examples/s]


In [6]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama-3.1-8b-instant"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [7]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello world. What math problem would you like help with today?


#### GSM8K 데이터셋 확인해보기

In [8]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [ ]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [22]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama-3.1-8b-instant",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer_raw = extract_final_answer(response)

            predicted_answer = None
            is_correct = False

            if predicted_answer_raw is not None and str(predicted_answer_raw).strip() != "":
                try:
                    clean_answer = re.sub(r"[^\d.]", "", str(predicted_answer_raw))
                    predicted_answer = float(clean_answer)
                    
                    diff = abs(predicted_answer - correct_answer)
                    is_correct = diff < 1e-5
                except (ValueError, TypeError):
                    predicted_answer = None
                    is_correct = False
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [23]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [16]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [17]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

 50%|█████     | 5/10 [00:10<00:15,  3.16s/it]

Progress: [5/10]
Current Acc.: [60.00%]


100%|██████████| 10/10 [00:28<00:00,  2.90s/it]

Progress: [10/10]
Current Acc.: [60.00%]


In [18]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!

for shot in [0, 3, 5]:
    prompt = construct_direct_prompt(shot)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=prompt,
        num_samples=50
    )
    save_final_result(results, accuracy, f"direct_prompting_{shot}.txt")
    print(f"Saved results for {shot}-shot prompting to direct_prompting_{shot}.txt")

 10%|█         | 5/50 [00:03<00:23,  1.90it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:05<00:17,  2.23it/s]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [00:08<00:18,  1.93it/s]

...

Progress: [40/50]
Current Acc.: [77.50%]


 90%|█████████ | 45/50 [03:40<00:23,  4.76s/it]

Progress: [45/50]
Current Acc.: [75.56%]


100%|██████████| 50/50 [04:04<00:00,  4.89s/it]

Progress: [50/50]
Current Acc.: [76.00%]
Saved results for 5-shot prompting to direct_prompting_5.txt


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [19]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    #TODO: 프롬프트를 작성해주세요!
    prompt = "Instruction:\nSolve the following mathematical questions with step-by-step reasoning.\n"

    for i in range(num_examples):
        #TODO: CoT example을 만들어주세요!
        cur_question = train_dataset['question'][i]
        full_answer = train_dataset['answer'][i].replace("####", "The final answer is:")
        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{full_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [20]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

for shot in [0, 3, 5]:
    prompt = construct_CoT_prompt(shot)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=prompt,
        num_samples=50
    )
    save_final_result(results, accuracy, f"CoT_prompting_{shot}.txt")
    print(f"Saved results for {shot}-shot CoT prompting to CoT_prompting_{shot}.txt")

 10%|█         | 5/50 [00:02<00:23,  1.90it/s]

Progress: [5/50]
Current Acc.: [100.00%]


 20%|██        | 10/50 [00:05<00:24,  1.66it/s]

Progress: [10/50]
Current Acc.: [80.00%]


 30%|███       | 15/50 [00:09<00:34,  1.03it/s]

...

Progress: [45/50]
Current Acc.: [84.44%]


100%|██████████| 50/50 [05:02<00:00,  6.06s/it]


Progress: [50/50]
Current Acc.: [84.00%]
Saved results for 3-shot CoT prompting to CoT_prompting_3.txt


  4%|▍         | 2/50 [00:33<13:33, 16.95s/it]


ValueError: could not convert string to float: ''

In [25]:
# 오류난 부분만 다시 돌리기

for shot in [5]:
    prompt = construct_CoT_prompt(shot)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=prompt,
        num_samples=50
    )
    save_final_result(results, accuracy, f"CoT_prompting_{shot}.txt")
    print(f"Saved results for {shot}-shot CoT prompting to CoT_prompting_{shot}.txt")

 10%|█         | 5/50 [00:03<00:27,  1.65it/s]

Progress: [5/50]
Current Acc.: [100.00%]


 20%|██        | 10/50 [00:21<03:09,  4.74s/it]

Progress: [10/50]
Current Acc.: [80.00%]


 30%|███       | 15/50 [01:12<05:23,  9.25s/it]

...

Progress: [40/50]
Current Acc.: [82.50%]


 90%|█████████ | 45/50 [05:04<00:38,  7.66s/it]

Progress: [45/50]
Current Acc.: [80.00%]


100%|██████████| 50/50 [05:41<00:00,  6.83s/it]

Progress: [50/50]
Current Acc.: [80.00%]
Saved results for 5-shot CoT prompting to CoT_prompting_5.txt


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [26]:
### 자유롭게 수정해도 됩니다! 완전히 새로 함수를 만들어도 돼요.
def construct_my_prompt(example_list: List[str], num_examples: int = 3):
    # TODO: 구현해주세요!
    train_dataset = gsm8k_train

    prompt = "Instruction:\n1. Analyze the question. 2. Solve it step-by-step. 3. Briefly double-check your calculation. 4. Provide the final numeric answer after 'Answer:'.\n"
    
    if num_examples > 0:
        sampled_indices = random.sample(range(len(train_dataset)), num_examples)
        for i, idx in enumerate(sampled_indices):
            q = train_dataset[idx]['question']
            raw_a = train_dataset[idx]['answer'].split("####")
            explanation = raw_a[0].strip()
            final_val = raw_a[1].strip()
            
            prompt += f"\n[Example {i+1}]\nQuestion: {q}\n"
            prompt += f"Steps: {explanation}\nVerification: Checked. The logic holds.\nAnswer: {final_val}\n"
    
    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [28]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!

for shot in [0]:
    prompt = construct_my_prompt([], shot)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=prompt,
        num_samples=50
    )
    save_final_result(results, accuracy, f"My_prompting_{shot}.txt")
    print(f"Saved results for {shot}-shot My prompting to My_prompting_{shot}.txt")

 10%|█         | 5/50 [00:03<00:31,  1.44it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:18<01:45,  2.64s/it]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [00:33<01:46,  3.03s/it]

...

Progress: [40/50]
Current Acc.: [80.00%]


 90%|█████████ | 45/50 [02:02<00:17,  3.54s/it]

Progress: [45/50]
Current Acc.: [77.78%]


100%|██████████| 50/50 [02:16<00:00,  2.73s/it]

Progress: [50/50]
Current Acc.: [80.00%]
Saved results for 0-shot My prompting to My_prompting_0.txt


In [ ]:
for shot in [3]:
    prompt = construct_my_prompt([], shot)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=prompt,
        num_samples=50
    )
    save_final_result(results, accuracy, f"My_prompting_{shot}.txt")
    print(f"Saved results for {shot}-shot My prompting to My_prompting_{shot}.txt")

In [ ]:
for shot in [5]:
    prompt = construct_my_prompt([], shot)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=prompt,
        num_samples=50
    )
    save_final_result(results, accuracy, f"My_prompting_{shot}.txt")
    print(f"Saved results for {shot}-shot My prompting to My_prompting_{shot}.txt")

### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!